In [28]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Basic Transformation - I")
    .master("local[*]")
    .getOrCreate()
)

spark

In [29]:
# Check default Parallelism
spark.sparkContext.defaultParallelism

16

In [30]:
# Read the data
logs_df = spark.read.format("parquet").load("/home/jupyter/raw_data/AWSLogs/713881793976/CloudFront/*")

In [31]:
logs_df.printSchema()

root
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- x_edge_location: string (nullable = true)
 |-- sc_bytes: string (nullable = true)
 |-- c_ip: string (nullable = true)
 |-- cs_method: string (nullable = true)
 |-- cs_Host: string (nullable = true)
 |-- cs_uri_stem: string (nullable = true)
 |-- sc_status: string (nullable = true)
 |-- cs_Referer: string (nullable = true)
 |-- cs_User_Agent: string (nullable = true)
 |-- cs_uri_query: string (nullable = true)
 |-- cs_Cookie: string (nullable = true)
 |-- x_edge_result_type: string (nullable = true)
 |-- x_edge_request_id: string (nullable = true)
 |-- x_host_header: string (nullable = true)
 |-- cs_protocol: string (nullable = true)
 |-- cs_bytes: string (nullable = true)
 |-- time_taken: string (nullable = true)
 |-- x_forwarded_for: string (nullable = true)
 |-- ssl_protocol: string (nullable = true)
 |-- ssl_cipher: string (nullable = true)
 |-- x_edge_response_result_type: string (nullable = true)
 |-

In [32]:
logs_df.count()

58931

In [24]:
from pyspark.sql.functions import col

logs_cols = logs_df.select("cs_protocol", "time_to_first_byte")

In [25]:
logs_cols.show()

+-----------+------------------+
|cs_protocol|time_to_first_byte|
+-----------+------------------+
|      https|             0.298|
|      https|             0.268|
|      https|             0.310|
|      https|             0.267|
|       http|             0.001|
|      https|             0.053|
+-----------+------------------+



In [27]:
logs_df.write.format("parquet").mode("append").save("/home/jupyter/clean_data")